# Library

In [1]:
from beb_chargers.scripts.script_helpers import build_trips_df, build_charger_location_inputs
from beb_chargers.opt.charger_location import ChargerLocationModel
from beb_chargers.gtfs_beb import GTFSData
from beb_chargers.vis import plot_trips_and_terminals, plot_deadhead
from pathlib import Path
import datetime
import pandas as pd
import logging
logging.basicConfig(level=logging.INFO)

import numpy as np
import matplotlib.pyplot as plt
import os

In [2]:
plt.rcParams["font.family"] = "Times New Roman"

# Parameters

In [3]:
# Define a reusable image save path
image_dir = r"C:\Users\chris\OneDrive - UW\Documents\GitHub\CET593project1\image"
result_dir = r"C:\Users\chris\OneDrive - UW\Documents\GitHub\CET593project1\results"

## site locations

In [4]:
alpha = 190 * 365 * 12 / 60

# coordinate location for overnight depot
depot_coords = (40.819197, -73.957060)

# power output
chg_pwrs = 450 / 60

# max number of chargers per site
n_max_list = [8, 4, 1]

# costs
s_cost_list = [200000, 500000, 800000]
c_cost_list = [600000, 800000, 1000000]

In [5]:
# location candidates
site_fname = Path.cwd().parent / 'beb_chargers' / 'data' / 'filtered_whole_station.csv'

# load candidates
loc_df = pd.read_csv(site_fname)
loc_df['kw'] = chg_pwrs * 60

In [6]:
"""
loc_df['max_chargers'] = n_max
loc_df['fixed_cost'] = s_cost
loc_df['charger_cost'] = c_cost
"""

"\nloc_df['max_chargers'] = n_max\nloc_df['fixed_cost'] = s_cost\nloc_df['charger_cost'] = c_cost\n"

## bus routes

In [7]:
# date of operation
ocl_date = datetime.datetime(2025, 3, 14)

# energy consumption
kwh_per_mi = 3

# battery_capacity
# battery_cap_arr = np.arange(100, 551, 25)
battery_cap_list = [850, 650, 550, 400, 300]

In [8]:
# Directory to GTFS files, as a platform-agnostic path
gtfs_dir = Path.cwd().parent / 'beb_chargers' / 'data' / 'gtfs' / 'GTFS'

# Load GTFS data into our custom object
gtfs = GTFSData.from_dir(gtfs_dir)

In [9]:
# All routes included in analysis
busy_routes = [
    'M15-SBS', 'Q58', 'B6', 'Bx12-SBS', 'B82-SBS', 'B46-SBS', 'B44-SBS', 'Q27', 'B41', 
    'Q44-SBS', 'B35', 'M14A-SBS', 'M14D-SBS', 'M101', 'Bx1', 'Bx2', 'Q46', 'M86-SBS', 'B1', 
    'B8', 'Q17', 'M4'
]

manhattan_busy_routes = [
    'M3', 'M34-SBS', 'M96', 'M102', 'M11', 'M79-SBS', 'M23-SBS'
]

beb_routes = busy_routes 
    # + manhattan_busy_routes

In [10]:

# Call helper function to build up DF
beb_trips = build_trips_df(
    gtfs=gtfs,
    date=ocl_date,
    routes=beb_routes,
    depot_coords=depot_coords,
    add_depot_dh=True,
    add_kwh_per_mi=False,
    add_durations=False,
    routes_60=[]
)

# Add a column giving energy consumption for each trip
beb_trips['kwh_per_mi'] = kwh_per_mi

In [11]:
# Record how many trips and blocks are active on our specified routes on this day
logging.info(
    '{}: There are {} total trips to be served by {} BEB blocks.'.format(
        ocl_date.strftime('%m/%d/%y'), len(beb_trips), beb_trips['block_id'].nunique()
    )
)

INFO:root:03/14/25: There are 7688 total trips to be served by 1491 BEB blocks.


In [12]:
# Create a map of the problem instance
inst_map = plot_trips_and_terminals(
    beb_trips, loc_df, gtfs.shapes_df, 'light'
)

# These config params make map downloads higher resolution
config = {
    'toImageButtonOptions': {
        'format': 'png',
        'scale': 3
    }
}
inst_map.show(config=config)
# image_fname = 'bus_route_map.png'
# save_path = os.path.join(image_dir, image_fname)
# inst_map.write_image(save_path)

In [13]:
def loc_df_func(n_max, s_cost, c_cost, loc_df=loc_df):
    loc_df['max_chargers'] = n_max
    loc_df['fixed_cost'] = s_cost
    loc_df['charger_cost'] = c_cost
    return loc_df

def build_charger_loc_inputs_func(battery_cap, loc_df, gtfs=gtfs, beb_trips=beb_trips, depot_coords=depot_coords):
    return build_charger_location_inputs(
            gtfs=gtfs,
            trips_df=beb_trips,
            chargers_df=loc_df,
            depot_coords=depot_coords,
            battery_cap=battery_cap
        )

def create_instance(opt_kwargs):
    return ChargerLocationModel(**opt_kwargs)

In [14]:
def opt_model(n_max, s_cost, c_cost, battery_cap, gtfs=gtfs, beb_trips=beb_trips, loc_df=loc_df, depot_coords=depot_coords):
    loc_df = loc_df_func(n_max, s_cost, c_cost, loc_df=loc_df)
    opt_kwargs = build_charger_loc_inputs_func(battery_cap, loc_df, gtfs=gtfs, beb_trips=beb_trips, depot_coords=depot_coords)
    clm = create_instance(opt_kwargs)
    print('Instance created')
    clm.solve(alpha=alpha, opt_gap=0, bu_kwh=battery_cap)
    print('Solved')
    save_path_csv = os.path.join(result_dir, f"result_{n_max:01d}_{s_cost:06d}_{c_cost:07d}_{battery_cap:03d}.csv")
    save_path_sumcsv = os.path.join(result_dir, f"result_summary_{n_max:01d}_{s_cost:06d}_{c_cost:07d}_{battery_cap:03d}.csv")
    clm.to_csv(save_path_csv)
    clm.summary_to_csv(save_path_sumcsv)
    print('File saved')
    tmp_df = clm.to_df()
    if tmp_df.shape[0] > 0:
        save_path_plot_chargers = os.path.join(image_dir, f"plot_chargers_{n_max:01d}_{s_cost:06d}_{c_cost:07d}_{battery_cap:03d}.png")
        clm.plot_chargers(save_path_plot_chargers)
        """
        fig = plot_deadhead(
            result_df=clm.to_df(), loc_df=loc_df, coords_df=beb_trips
        )
        fig.show()
        """
    else:
        print('empty dataframe')

In [15]:
# opt_model(10, 200000, 600000, 550, gtfs=gtfs, beb_trips=beb_trips, loc_df=loc_df, depot_coords=depot_coords)

In [ ]:
for battery_cap in battery_cap_list:
    for n_max in n_max_list:
        for s_cost in s_cost_list:
            for c_cost in c_cost_list:
                opt_model(n_max, s_cost, c_cost, battery_cap, gtfs=gtfs, beb_trips=beb_trips, loc_df=loc_df, depot_coords=depot_coords)

INFO:charger_location:Number of blocks that require charging: 0
INFO:charger_location:Number of trips in charging blocks: 0
INFO:charger_location:Number of infeasible blocks: 0


Instance created


INFO:gurobipy:Set parameter Username
INFO:gurobipy:Set parameter LicenseID to value 2608727
INFO:gurobipy:Academic license - for non-commercial use only - expires 2026-01-12
INFO:gurobipy:Read LP format model from file C:\Users\chris\AppData\Local\Temp\tmpswwaphpp.pyomo.lp
INFO:gurobipy:Reading time = 0.01 seconds
INFO:gurobipy:x1: 4 rows, 8 columns, 8 nonzeros
INFO:gurobipy:Set parameter MIPGap to value 0
INFO:gurobipy:Set parameter MIPFocus to value 3
INFO:gurobipy:Set parameter Method to value 1
INFO:gurobipy:Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (26100.2))
INFO:gurobipy:
INFO:gurobipy:CPU model: 13th Gen Intel(R) Core(TM) i5-13450HX, instruction set [SSE2|AVX|AVX2]
INFO:gurobipy:Thread count: 10 physical cores, 16 logical processors, using up to 16 threads
INFO:gurobipy:
INFO:gurobipy:Non-default parameters:
INFO:gurobipy:MIPGap  0
INFO:gurobipy:Method  1
INFO:gurobipy:MIPFocus  3
INFO:gurobipy:
INFO:gurobipy:Optimize a model with 4 rows, 8 columns 

Solved
File saved
empty dataframe


INFO:charger_location:Number of blocks that require charging: 0
INFO:charger_location:Number of trips in charging blocks: 0
INFO:charger_location:Number of infeasible blocks: 0


...

Solved
File saved
empty dataframe


INFO:charger_location:Number of blocks that require charging: 0
INFO:charger_location:Number of trips in charging blocks: 0
INFO:charger_location:Number of infeasible blocks: 0


Instance created


INFO:gurobipy:Read LP format model from file C:\Users\chris\AppData\Local\Temp\tmp5clv_tiu.pyomo.lp
INFO:gurobipy:Reading time = 0.01 seconds
INFO:gurobipy:x1: 4 rows, 8 columns, 8 nonzeros
INFO:gurobipy:Set parameter MIPGap to value 0
INFO:gurobipy:Set parameter MIPFocus to value 3
INFO:gurobipy:Set parameter Method to value 1
INFO:gurobipy:Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (26100.2))
INFO:gurobipy:
INFO:gurobipy:CPU model: 13th Gen Intel(R) Core(TM) i5-13450HX, instruction set [SSE2|AVX|AVX2]
INFO:gurobipy:Thread count: 10 physical cores, 16 logical processors, using up to 16 threads
INFO:gurobipy:
INFO:gurobipy:Non-default parameters:
INFO:gurobipy:MIPGap  0
INFO:gurobipy:Method  1
INFO:gurobipy:MIPFocus  3
INFO:gurobipy:
INFO:gurobipy:Optimize a model with 4 rows, 8 columns and 8 nonzeros
INFO:gurobipy:Model fingerprint: 0x12fc0271
INFO:gurobipy:Variable types: 0 continuous, 8 integer (4 binary)
INFO:gurobipy:Coefficient statistics:
INFO:gurobip

Solved
File saved
empty dataframe
